In [63]:
import argparse
import numpy as np
import pandas as pd
import glob
import math
import os
import shutil
import time
import torch
import torch.nn as nn
import torch.utils.data as loader
import torch.optim as optim
from torchvision import models
from torchvision import transforms
import torchvision.datasets as datasets
from typing import List
from PIL import Image
import requests
import matplotlib.pyplot as plt
import urllib.request

In [64]:
# !tar -xvzf ./101_ObjectCategories.tar.gz 

### Data Splitting

In [65]:
def ls(path):
    files = os.listdir(path)
    return np.asarray(files)


def split_data(prev, curr, classes):
    for name in classes:
        curr_dir = os.path.join(os.getcwd(), f"{prev}/{name}")
        
        files = ls(curr_dir)
        total_files = np.size(files, 0)
        print(f'Total files : {total_files}')
        
        # Training data 75%
        train_size = math.ceil(total_files * 3/4)
        print(f'Total files : {train_size}')
        
        # Validation data 12.5
        validation_size = train_size + math.ceil(total_files * 1/8) #used for indexing so
        print(f'Total files : {validation_size - train_size}')
        
        # Test data 12.5%
        test_size = validation_size + math.ceil(total_files * 1/8)
        print(f'Total files : {test_size - validation_size}')
        
        train = files[0:train_size]
        validation = files[train_size:validation_size]
        test = files[validation_size:]
        
        # Creating directories for all data splits and moving them accordingly
        mv(train, curr_dir, f'train/{name}')
        mv(validation, curr_dir, f'validation/{name}')
        mv(test, curr_dir, f'test/{name}')
        
        
def mv(files, prev, curr):
    curr = os.path.join(os.getcwd(), curr);
    if not os.path.exists(curr):
        os.makedirs(curr)
        
    for file in np.nditer(files):
        prev_path = os.path.join(os.getcwd(), f'{prev}/{file}')
        curr_path = os.path.join(os.getcwd(), f'{curr}/{file}')
        
        shutil.move(prev_path, curr_path)

### Input Transformation

In [66]:
def input_transform():
    image_transforms = {
        'train': transforms.Compose([
                  transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
                  transforms.RandomRotation(degrees=15),
                  transforms.RandomHorizontalFlip(),
                  transforms.CenterCrop(size=224),
                  transforms.ToTensor(),
                  transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ]),
        'validation': transforms.Compose([
                 transforms.Resize(size=256),
                 transforms.CenterCrop(size=224),
                 transforms.ToTensor(),
                 transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])              
        ]),
        'test': transforms.Compose([
                 transforms.Resize(size=256),
                 transforms.CenterCrop(size=224),
                 transforms.ToTensor(),
                 transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])              
        ])
    }
    return image_transforms

### Data Loader

In [67]:
def load_data():
    data = {
        'train': datasets.ImageFolder(root='./train', transform=image_transforms['train']),
        'validation': datasets.ImageFolder(root='./validation', transform=image_transforms['validation']),
        'test': datasets.ImageFolder(root='./test', transform=image_transforms['test'])
    }
    return data

### Model

In [68]:

def def_model():
    model = models.resnet50(pretrained=True)
     
    for param in model.parameters():
      param.requires_grad = False

    fc_inputs = model.fc.in_features

    model.fc = nn.Sequential(
        nn.Linear(fc_inputs, 2048),
        nn.ReLU(inplace=True),
        nn.Linear(2048, 10),
        nn.Dropout(0.4),
        nn.LogSoftmax(dim=1)
    )
    
    if(torch.cuda.is_available()):
        model = model.to('cuda:0')
        
    return model

### Optimization of loss/cost function

In [69]:
def opt_loss(model):
    loss_func = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters())
    return loss_func, optimizer

### Model Fitting

In [70]:
def train_and_validate(model, loss_criterion, optimizer, epochs=25):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  start = time.time()
  history = []
  best_acc = 0.0
  for epoch in range(epochs):
    epoch_start = time.time()
    print(f'Epoch : {epoch+1}/{epochs}')
    model.train()
    train_loss = 0.0 
    train_acc = 0.0

    valid_loss = 0.0
    valid_acc  = 0.0

    for i, (inputs, labels) in enumerate(train_data):
      inputs = inputs.to(device)
      labels = labels.to(device)
      # Clean existing gradients
      optimizer.zero_grad()
      # Forward pass - compute outputs on input data using the model
      outputs = model(inputs)
      # Compute loss
      loss = loss_criterion(outputs, labels)
      # Backpropagate the gradients
      loss.backward()
      # Update the parameters
      optimizer.step()
      # Compute the total loss for the batch and it to train_loss
      train_loss += loss.item() * inputs.size(0)
      # Compute the accuracy
      ret, predictions = torch.max(outputs.data,1)
      correct_counts = predictions.eq(labels.data.view_as(predictions))

      # Convert correct_counts to float and then compute the mean
      acc = torch.mean(correct_counts.type(torch.FloatTensor))
      # Compute total accuracy in the whole batch and add to train_acc
      train_acc += acc.item() * inputs.size(0)
      print(f'Batch number: {i}, Training: Loss: {loss.item()}, Accuracy: {acc.item()}')

    with torch.no_grad():
      model.eval()
      for j, (inputs, labels) in enumerate(validation_data): 
        inputs = inputs.to(device)
        labels = labels.to(device)
        # Forward pass - compute outputs on input data using the model
        outputs = model(inputs)

        # Compute loss
        loss = loss_criterion(outputs, labels)
        # Compute the total loss for  the batch and add it to valid_loss
        valid_loss += loss.item() * inputs.size(0)
        # Calculate validation accuracy

        ret, predictions = torch.max(outputs.data, 1)
        correct_prediction_counts = predictions.eq(labels.data.view_as(predictions))

        # Convert correct_prediction_counts to float and then compute the mean
        acc = torch.mean(correct_prediction_counts.type(torch.FloatTensor))

        # Compute total accuracy in the whole batch and add to valid_acc

        valid_acc +=acc.item() * inputs.size(0)

      avg_train_loss = train_loss/train_data_size
      avg_train_acc = train_acc/train_data_size

      avg_valid_loss = valid_loss/validation_data_size
      avg_valid_acc = valid_acc/validation_data_size

      history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])

      epoch_end = time.time()

      print(f'Epoch : {epoch}, Training: Loss: f{avg_train_loss}, Accuracy: {avg_train_acc*100}%, \n\t\tValidation : Loss : {avg_valid_loss}, Accuracy: {avg_valid_acc*100}%, Time: {epoch_end-epoch_start}s')

      # Save if the model has best accuracy till now

      torch.save(model.state_dict(), f'model_{epoch}.pth')

    return model, history

### Model accuracy

In [71]:
def computeModelAccuracy(model, loss_criterion):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  test_acc = 0.0
  test_loss = 0.0

  with torch.no_grad():
    # Set to evaluation mode
    model.eval()
    for i, (inputs, labels) in enumerate(test_data):
      inputs = inputs.to(device)
      labels = labels.to(device)
      outputs = model(inputs)

      # Compute loss
      loss = loss_criterion(outputs, labels)

      # Compute the toal loss item 
      test_loss += loss.item() * inputs.size(0)

      ret, predictions = torch.max(outputs.data, 1)
      correct_counts = predictions.eq(labels.data.view_as(predictions))
      acc = torch.mean(correct_counts.type(torch.FloatTensor))
      test_acc +=acc.item() * inputs.size(0)

      print(f'Test Batch number: {i}, Test: Loss: {loss.item()}, Accuracy: {acc.item()}')

      # Find average test loss and test accuracy
      avg_test_loss = test_loss/test_data_size
      avg_test_acc = test_acc/test_data_size

      print(f'Test accuracy: {avg_test_acc}')

### Model prediction and image processing

In [72]:
def makePrediction(model, url):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    transform = image_transforms['test']

    test_image = Image.open(requests.get(url, stream=True).raw)

    plt.imshow(test_image)

    test_image_tensor = transform(test_image)
    test_image_tensor = test_image_tensor.view(1, 3, 224, 224).to(device)
    with torch.no_grad():
        model.eval()
        out = model(test_image_tensor)
        ps = torch.exp(out)

        topk, topclass = ps.topk(3, dim=1)
        for i in range(3):
            print(f"Prediction {i+1} : {index_to_class[topclass.cpu().numpy()[0][i]]}, Score: {topk.cpu().numpy()[0][i] * 100}%")

## Main

In [73]:
# def main():
#     # split dat
#     classes = ['Leopards', 'airplanes', 'butterfly', 'camera', 'elephant', 'lamp', 'watch', 'umbrella', 'rhino'];
#     split_data('101_ObjectCategories', './', classes)
    
#     # transform data
#     input_transform()
    
#     # created dataset
#     batch_size = 10
#     data = load_data()
    
#     # create a data loader instance with each dataset with a batch size of 10 and shuffling
#     train_data = loader.DataLoader(data['train'], batch_size=batch_size, shuffle=True)
#     validation_data = loader.DataLoader(data['validation'], batch_size=batch_size, shuffle=True)
#     test_data = loader.DataLoader(data['test'], batch_size=batch_size, shuffle=True)
    
#     #data size
#     train_data_size = len(data['train'])
#     validation_data_size = len(data['validation'])
#     test_data_size =  len(data['test'])
    
#     # get model and change r50 model output to classify
#     model = def_model()
    
#     # select optimiser and define loss func
#     loss_func, optimizer = opt_loss(model)
    
#     # fitting / backpropagation
#     num_epochs = 30
#     trained_model, history = train_and_validate(model, loss_func, optimizer, num_epochs)
    
#     # accuracy on testdata
#     model = models.resnet50(pretrained=False)
#     fc_inputs = model.fc.in_features

#     model.fc = nn.Sequential(
#         nn.Linear(fc_inputs, 2048),
#         nn.ReLU(inplace=True),
#         nn.Linear(2048, 10),
#         nn.Dropout(0.4),
#         nn.LogSoftmax(dim=1))

#     if(torch.cuda.is_available()):
#         model = model.to("cuda")
#     model.load_state_dict(torch.load('model_0.pth'))

#     computeModelAccuracy(model, loss_func)
    
#     # testing 
#     index_to_class = {v: k for k, v in data['train'].class_to_idx.items()}
#     print (index_to_class)

#     makePrediction(model, 'https://cdn.britannica.com/30/136130-050-3370E37A/Leopard.jpg')
# #     model = torch.load('_model_0.pt')

In [74]:
# if __name__ == "__main__":
#     main()

In [75]:
    # split dat
    classes = ['Leopards', 'airplanes', 'butterfly', 'camera', 'elephant', 'lamp', 'watch', 'umbrella', 'rhino'];
    split_data('101_ObjectCategories', './', classes)
    
    # transform data
    image_transforms = input_transform()
    
    # created dataset
    batch_size = 10
    data = load_data()
    
    # create a data loader instance with each dataset with a batch size of 10 and shuffling
    train_data = loader.DataLoader(data['train'], batch_size=batch_size, shuffle=True)
    validation_data = loader.DataLoader(data['validation'], batch_size=batch_size, shuffle=True)
    test_data = loader.DataLoader(data['test'], batch_size=batch_size, shuffle=True)
    
    #data size
    train_data_size = len(data['train'])
    validation_data_size = len(data['validation'])
    test_data_size =  len(data['test'])
    
    # get model and change r50 model output to classify
    model = def_model()
    
    # select optimiser and define loss func
    loss_func, optimizer = opt_loss(model)
    
    # fitting / backpropagation
    num_epochs = 30
    trained_model, history = train_and_validate(model, loss_func, optimizer, num_epochs)
    
    # accuracy on testdata
    model = models.resnet50(pretrained=False)
    fc_inputs = model.fc.in_features

    model.fc = nn.Sequential(
        nn.Linear(fc_inputs, 2048),
        nn.ReLU(inplace=True),
        nn.Linear(2048, 10),
        nn.Dropout(0.4),
        nn.LogSoftmax(dim=1))

    if(torch.cuda.is_available()):
        model = model.to("cuda")
    model.load_state_dict(torch.load('model_0.pth'))

    computeModelAccuracy(model, loss_func)
    
    # testing 
    index_to_class = {v: k for k, v in data['train'].class_to_idx.items()}
    print (index_to_class)

Total files : 200
Total files : 150
Total files : 25
Total files : 25
Total files : 800
Total files : 600
Total files : 100
Total files : 100
Total files : 91
Total files : 69
Total files : 12
Total files : 12
Total files : 50
Total files : 38
Total files : 7
Total files : 7
Total files : 64
Total files : 48
Total files : 8
Total files : 8
Total files : 61
Total files : 46
Total files : 8
Total files : 8
Total files : 239
Total files : 180
Total files : 30
Total files : 30
Total files : 75
Total files : 57
Total files : 10
Total files : 10
Total files : 59
Total files : 45
Total files : 8
Total files : 8


C:\Users\91628\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\91628\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch : 1/30
Batch number: 0, Training: Loss: 2.317222833633423, Accuracy: 0.0
Batch number: 1, Training: Loss: 6.524583339691162, Accuracy: 0.20000000298023224
Batch number: 2, Training: Loss: 3.7636077404022217, Accuracy: 0.5
Batch number: 3, Training: Loss: 4.386628150939941, Accuracy: 0.30000001192092896
Batch number: 4, Training: Loss: 1.8683111667633057, Accuracy: 0.699999988079071
Batch number: 5, Training: Loss: 2.348623752593994, Accuracy: 0.10000000149011612
Batch number: 6, Training: Loss: 3.3893685340881348, Accuracy: 0.10000000149011612
Batch number: 7, Training: Loss: 2.557569980621338, Accuracy: 0.20000000298023224
Batch number: 8, Training: Loss: 2.283421277999878, Accuracy: 0.30000001192092896
Batch number: 9, Training: Loss: 1.911400556564331, Accuracy: 0.30000001192092896
Batch number: 10, Training: Loss: 1.563279390335083, Accuracy: 0.5
Batch number: 11, Training: Loss: 1.4541476964950562, Accuracy: 0.6000000238418579
Batch number: 12, Training: Loss: 1.872816324234

Batch number: 102, Training: Loss: 1.4027235507965088, Accuracy: 0.699999988079071
Batch number: 103, Training: Loss: 1.1437294483184814, Accuracy: 0.800000011920929
Batch number: 104, Training: Loss: 1.0934722423553467, Accuracy: 0.800000011920929
Batch number: 105, Training: Loss: 0.7348465919494629, Accuracy: 0.699999988079071
Batch number: 106, Training: Loss: 0.681165337562561, Accuracy: 0.800000011920929
Batch number: 107, Training: Loss: 0.726866602897644, Accuracy: 0.699999988079071
Batch number: 108, Training: Loss: 0.5208412408828735, Accuracy: 0.800000011920929
Batch number: 109, Training: Loss: 0.6325676441192627, Accuracy: 0.800000011920929
Batch number: 110, Training: Loss: 0.907440185546875, Accuracy: 0.699999988079071
Batch number: 111, Training: Loss: 0.8558406829833984, Accuracy: 0.699999988079071
Batch number: 112, Training: Loss: 0.9228814840316772, Accuracy: 0.699999988079071
Batch number: 113, Training: Loss: 1.067461371421814, Accuracy: 0.5
Batch number: 114, Tra

C:\Users\91628\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Test Batch number: 0, Test: Loss: 0.049381356686353683, Accuracy: 1.0
Test accuracy: 0.050505050505050504
Test Batch number: 1, Test: Loss: 0.022839510813355446, Accuracy: 1.0
Test accuracy: 0.10101010101010101
Test Batch number: 2, Test: Loss: 0.28585997223854065, Accuracy: 0.8999999761581421
Test accuracy: 0.14646464526051223
Test Batch number: 3, Test: Loss: 0.04226775839924812, Accuracy: 1.0
Test accuracy: 0.19696969576556272
Test Batch number: 4, Test: Loss: 0.14028964936733246, Accuracy: 1.0
Test accuracy: 0.24747474627061325
Test Batch number: 5, Test: Loss: 0.042556945234537125, Accuracy: 1.0
Test accuracy: 0.2979797967756637
Test Batch number: 6, Test: Loss: 0.1610698401927948, Accuracy: 0.8999999761581421
Test accuracy: 0.34343434102607495
Test Batch number: 7, Test: Loss: 0.15353187918663025, Accuracy: 1.0
Test accuracy: 0.39393939153112545
Test Batch number: 8, Test: Loss: 0.1850665956735611, Accuracy: 0.8999999761581421
Test accuracy: 0.4393939357815367
Test Batch number: 